<a href="https://colab.research.google.com/github/harveyrutland/swarm_routines/blob/main/swarm_boatproperties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

def adjust_position_perpX(boat,fin,dist): 
  angle = get_angle(boat,fin)
  boatx_delta = 3.25 * math.cos(angle)
  if boatx_delta > 0:
    boatx_delta = 0
  return boatx_delta


def adjust_position_perpY(deltX, angle):
  y_adjustment = math.tan(angle) * deltX
  return y_adjustment
0.4

def return_dist(boatx, boaty, leaderx, leadery):
  x = [boatx, leaderx]
  y = [boaty, leadery]
  delta_x = abs(boatx - leaderx)
  delta_y = abs(boaty - leadery)
  hypoteneus = math.sqrt(delta_x**2 + delta_y**2)
  return hypoteneus

def get_key_from_value(d, val):
  keys = [k for k, v in d.items() if v == val]
  if keys:
      return keys[0]
  return None

def get_angle(point_1, point_2): 
    angle = atan2(point_1[1] - point_2[1], point_1[0] - point_2[0])
    return angle

def find_steps(distance):
  constant = 0.4
  steps = distance / constant
  steps = int(round(steps))
  # print('step number:', steps)
  if steps < 2:
    steps = 2
  return steps


def move_to_leader(boat1_startx, boat1_starty, leaderx, leadery, step):
    boat_path = np.linspace([boat1_startx,boat1_starty],[leaderx, leadery], step)
    newboatloc = (boat_path[1][0], boat_path[1][1])
    return (newboatloc)


class SetEnv:
  def __init__(self):
    self.rect_startx = random.uniform(4, 5)
    
#############################
#############################
    #search paramters 
    self.rect_starty = 1
    self.box_size = 2
#############################
#############################


  def print_boats(self):
    print(self.num)
  
  def clear_frames(self):
    #frame of animation
    ax.clear()
    ax.set_xlim(( 0, 30))
    ax.set_ylim((-6,6))
  
  def add_start_box(self):
    #start square 
    ax.add_patch(patches.Rectangle((self.rect_startx, self.rect_starty),self.box_size, self.box_size,fill=False))
    return self.rect_startx, self.rect_starty
    #frames
    # p = plt.Polygon(np.c_[x,y], color='white')
    # ax.add_patch(p)


  


#######
class AddBoats(SetEnv):
   def __init__(self, set_env):
        self.ascending = True
        self.seen_boat1 = False
        self.search_state = True 
        self.seen_prior = False
        self.register_shark_lost = False
        self.follower_step = 5
        self.anchor_step = 7
        self.follower_angle = 0
        self.inward_state = None
        
        self.PID_angle = 0
        self.pid = PID(20, 12.5, 2, setpoint= 2)
        self.pid.output_limits = (-60, 60) 
        self.initial_threshold = 3



#############################
#############################
        #search paramters 
        self.view_point_delta = None
        self.distance_to_trigger_movement = None
#############################
#############################
        
        
        


        # boat view point and and angle variables 
        self.angle = random.uniform(0, 360)  
        # self.angle = math.pi + math.pi/3

        # random angle view point rotation limits are set with -1.5 to 1.5 rad as upper and lower limits self.angle is subtracted to adjust to position of boat 
        self.boat_rotation = self.angle
        self.radjustment = 0 



        # self.rotation = math.radians(self.angle) 
        self.rotation = self.angle
        #sets the correct boat starting position relative to 0 
        self.boat_rotation_offset = 0
        self.perp_offsetdjustment = 0 
        self.dist = 0 


        #---- variable to dtermine direction in which shark is travelling ----# 
        self.last_angle = 0
        self.first_angle = 0 


        self.follow_shark_perp = False
        self.direction_check_interval = 0


        #change back to start random
        self.boat1_startx = random.uniform(set_env.rect_startx, set_env.rect_startx+1)
        self.boat1_starty = random.uniform(set_env.rect_starty, set_env.rect_starty+1)

  
   def set_rotation(self, x_fin, y_fin):

     if self.search_state == True:
       if self.ascending == True:    #  self.boat1_startx, self.boat1_starty = move_to_leader(self.boat1_startx, self.boat1_starty, leaderx, leadery)
         self.rotation += 0.2
         if self.rotation >= (1.5 + self.angle):
           self.ascending = False
        
       if self.ascending == False:
         self.rotation += -0.2
         if self.rotation <= (-1.5 + self.angle):
           self.ascending = True 

     elif self.search_state == False:
        self.rotation = get_angle([x_fin, y_fin], [self.boat1_startx, self.boat1_starty])

     self.xoffset1 = math.cos(self.rotation)*5
     self.yoffset1 = math.sin(self.rotation)*5
     self.xoffsetL = math.cos(self.rotation - 0.5)*8
     self.yoffsetL = math.sin(self.rotation - 0.5)*8
     self.xoffsetR = math.cos(self.rotation + 0.5)*8
     self.yoffsetR = math.sin(self.rotation + 0.5)*8

     

     self.xoffset2 = math.cos(self.boat_rotation + self.perp_offsetdjustment + math.radians(self.boat_rotation_offset))*0.5
     self.yoffset2 = math.sin(self.boat_rotation + self.perp_offsetdjustment + math.radians(self.boat_rotation_offset))*0.5
   
  

   def check_search_state(self, x_fin, y_fin):

     if self.seen_boat1 == True:
        self.first_angle = get_angle([x_fin, y_fin], [self.boat1_startx, self.boat1_starty])
        if abs(degrees(self.first_angle)) > 0 :
          self.search_state = False


   def follow_shark(self, x_fin, y_fin):
 
      if self.seen_boat1 == True:
        

        self.first_angle = get_angle([x_fin, y_fin], [self.boat1_startx, self.boat1_starty])
        if abs(degrees(self.first_angle)) > (90-self.view_point_delta) and abs(degrees(self.first_angle)) < (90+self.view_point_delta) :
          self.seen_prior = True 
          self.search_state = False
          self.dist = return_dist(self.boat1_startx, self.boat1_starty, x_fin[0], y_fin[0])
          
          if self.dist >= self.distance_to_trigger_movement and self.follow_shark_perp == False:
            # self.boat1_startx, self.boat1_starty = move_to_shark(self.boat1_startx, self.boat1_starty, x_fin[0], y_fin[0])
            self.boat_rotation = self.rotation
            boat_path = np.linspace([self.boat1_startx, self.boat1_starty],[x_fin[0], y_fin[0]], 10)
            self.boat1_startx, self.boat1_starty = (boat_path[1][0], boat_path[1][1])
          
          if self.dist < self.distance_to_trigger_movement and self.follow_shark_perp == False:
            # write_to_log('Distance between boat and shark is:', dist)
            self.direction = 0
            self.direction_check_interval +=1
            
            if self.first_angle > self.last_angle:
              self.direction = 0
            elif self.first_angle < self.last_angle:
              self.direction = 1
            self.last_angle = self.first_angle 
            if self.direction_check_interval == 2:
              angle_changes = [90, -90]
              self.boat_rotation_offset = angle_changes[self.direction]

       
              self.follow_shark_perp = True
              # write_to_log('following perpenexceptdicular to shark')
              self.direction_check_interval = 0
     
      else:

        self.search_state = True 
        if self.seen_prior == True :
          self.register_shark_lost = True
          



      #follow shark perpendicular  
      if self.follow_shark_perp == True:
        
        self.direction_check_interval += 1
        boat_deltaY = 0
        if self.direction_check_interval == 1:

          self.boat_deltaX = adjust_position_perpX([self.boat1_startx, self.boat1_starty],[x_fin, y_fin], self.dist)
          
          
          init_angle = self.boat_rotation + self.perp_offsetdjustment + math.radians(self.boat_rotation_offset)
          self.follow_anlge = init_angle 
          radjustment = adjust_position_perpY(self.boat_deltaX , init_angle)
          

          self.dist == 
          control = self.pid(self.dist)
          self.perp_offsetdjustment = math.radians(control)
          boat_deltaY = self.boat_deltaX * math.tan(radians(-control))
          # print('Dist and controll', self.dist, control)

  

          self.boat1_startx = self.boat1_startx + abs(self.boat_deltaX)
          # self.boat1_starty = self.boat1_starty + boat_deltaY - radjustment
          self.boat1_starty = self.boat1_starty + boat_deltaY - radjustment
          self.direction_check_interval = 0
      
      return self.register_shark_lost, self.seen_prior, self.follow_shark_perp



  
      
  
   def follow_leader(self, leaderx, leadery):
     self.dist = return_dist(self.boat1_startx, self.boat1_starty, leaderx, leadery)

     if self.dist >= 1:
        self.boat1_startx, self.boat1_starty = move_to_leader(self.boat1_startx, self.boat1_starty, leaderx, leadery, self.follower_step)
        self.boat_rotation = get_angle([leaderx, leadery], [self.boat1_startx, self.boat1_starty])
        self.angle = self.boat_rotation 
        return  [self.boat1_startx, self.boat1_starty]
     else:
        return  [self.boat1_startx, self.boat1_starty]

    #    print('dist too small')
  
   def initaite_swarm(self, feederx, feedery, leaderx, leadery):     

     self.dist = return_dist(feederx, feedery, leaderx, leadery)
     distance_to_new_loc = return_dist(self.boat1_startx, self.boat1_starty, feederx, feedery)
     self.anchor_step = find_steps(distance_to_new_loc)
     self.boat_rotation = self.follower_angle
     
     if  self.dist <= self.initial_threshold:
       reset_anchor = True 
       self.boat1_startx, self.boat1_starty = move_to_leader(self.boat1_startx, self.boat1_starty, feederx, feedery, self.anchor_step)
       self.initial_threshold = 5
       return reset_anchor 
     
     else:
       reset_anchor = False
       self.boat1_startx, self.boat1_starty = move_to_leader(self.boat1_startx, self.boat1_starty, feederx, feedery, self.anchor_step)
       return reset_anchor 

          

  
   def plot_boats(self, x_fin, y_fin, boatnumber):
      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffsetL], [self.boat1_starty, self.boat1_starty + self.yoffsetL] , 'bo', linestyle="--", markevery=[0], markersize=2)
      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffsetR], [self.boat1_starty, self.boat1_starty + self.yoffsetR] , 'bo', linestyle="--", markevery=[0], markersize=2)
      x = np.append([self.boat1_startx, self.boat1_startx + self.xoffsetL], [self.boat1_startx, self.boat1_startx + self.xoffsetR][::-1])
      y = np.append([self.boat1_starty, self.boat1_starty + self.yoffsetL],[self.boat1_starty, self.boat1_starty + self.yoffsetR] [::-1])
      p = plt.Polygon(np.c_[x,y], color="lemonchiffon")
      ax.add_patch(p)
      view_region = Polygon([(self.boat1_startx, self.boat1_starty), (self.boat1_startx + self.xoffsetL, self.boat1_starty + self.yoffsetL),(self.boat1_startx + self.xoffsetR, self.boat1_starty + self.yoffsetR)])
      point = Point(x_fin, y_fin)

      self.seen_boat1 = view_region.contains(point)
      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffset2], [self.boat1_starty, self.boat1_starty + self.yoffset2] , 'go', linestyle="-", markevery=[0], markersize=2, linewidth=2)

      ax.plot([self.boat1_startx, self.boat1_startx + self.xoffset1],
              [self.boat1_starty, self.boat1_starty + self.yoffset1] , 
              'ro', linestyle="--", 
              markevery=[0],marker=(3, 0, (35 + math.degrees(self.boat_rotation + self.perp_offsetdjustment)+ self.boat_rotation_offset)),  markersize=10)
      return self.seen_boat1, boatnumber, [self.boat1_startx, self.boat1_starty]





class LeaderFollower():
  def __init__(self):
    self.leader_number = None
    self.leader_assigned = False
    self.shark_lost_report = None 
    self.boat_seen = None
    self.boat_seen = None
    self.leader_coords = None
    self.swarm_dist_to_leader = {}
    self.follower_assigned = False
    self.follower_boat = None

  def set_leader(self, seen_status, leader_status, boatnumber):
    if seen_status == True and leader_status == False:
      self.leader_assigned = True
      self.leader_number = boatnumber
      return self.leader_number, self.leader_assigned
    else: 
      return self.leader_number, self.leader_assigned

  
  def triangulate_anchor(self, leader_coords, follower_coords, gap):
    angle_leaderfollower = get_angle(follower_coords, leader_coords)
    delta_X = math.cos((math.pi-angle_leaderfollower)) * gap 
    delta_Y = math.sin((math.pi-angle_leaderfollower)) * gap
    anchor_coords = [leader_coords[0] + delta_X, leader_coords[1] - delta_Y]
    return anchor_coords
 

  def triangulate_feeder(self, leader_coords, follower_coords, gap, distance):
    angle_leaderfollower = get_angle(follower_coords, leader_coords)
    angle_leaderfollower = math.pi-angle_leaderfollower
    try:
      angle_adjustment = math.pi/2 - (math.acos(distance/gap))
    except ValueError:
      angle_adjustment = 0

    angle_leaderfollower = angle_leaderfollower + angle_adjustment
    delta_X = math.cos((angle_leaderfollower)) * gap 
    delta_Y = math.sin((angle_leaderfollower)) * gap
    feeder_coords = [leader_coords[0] + delta_X, leader_coords[1] - delta_Y]

    return feeder_coords
    



  





  
